# Template Definition

In [1]:
ROOT_FOLDER = ""

In [2]:
prompt_template_noun   = """
Step 1:

Identify all the nouns in the specification text which can potentially be the class name.
Include as much as nouns as possible and do not care about their functions for now.

###########

The specification text is:

{text}

##############

The class list is:
    
"""
prompt_template_class = """
Step 2:

You will be asked to extact, from text specification, the list of class to use in 
a uml conceptual model diagram. Include only classes relevant for conceptual modeling.


Identify classes from the nouns list extracted from the problem description above.
A class is the description for a set of similar objects that have the same structure and behavior, i.e., its instances
All objects with the same features and behavior are instances of one class.
In general, something should be a class if it could have instances.
In general, something should be an instance if it is clearly a single member of the set defined by a class.
Keep in mind that some of the nouns may be attributes or roles of the identified classes.
Choose proper names for classes according the the following rules:
1. Noun
2. Singular
3. Not too general, not too specific – at the right level of abstraction
4. Avoid software engineering terms (data, record, table, information)
5. Conventions: first letter capitalized; camel case without spaces if needed

Example class names:
Hospital, Doctor, PartTimeEmployee

Constraints:
Create classes at the right level of abstraction.
Not all nouns in the nouns list are classes, some of them may be attributes, role names, or even not needed for diagram.
Do NOT include all the nouns list as classes. Evaluate if it is needed to be a class.
ONLY generate classes that are necessary to develop the system.


Example:
Problem Description: This system helps the Java Valley police officers keep track of the cases they are assigned to do. 
Officers may be assigned to investigate particular crimes, 
which involves interviewing victims at their homes and entering notes in the PI system. 
Identified Class List: PoliceStation, Case, PoliceOfficer, Victim, Crime, Note


Output only the list and no other text.
Keep the list as short as possible and do not add irrelevant classes.

##############

The noun list is: 

{noun}

##############

The text specification is: 

{text}

##############

The class list is:
"""

prompt_template_assoc =  """
Step 3:

You will be asked to extact, from text specification and a list of classes, the associations amongst
the classes to use in a uml conceptual model diagram. 

Include only associations relevant for conceptual modeling. Use only classes from the list.

example format: mul1 Class1 -- mul2 Class2

Class1 and Class2 are classes above. mul1 and mul2 are one of the following options[0..*, 1..1, 0..1, 1..*]
there might be multiple associations
For example:
0..* Student -- 0..5 Registration
1..1 Student -- 0..1 StudentProfile

Note:
1. Use the classes in the given generated classes list, generate the classes and their relationships.
2. Only add the system class if the existing class diagram misses the system class.
3. Do NOT change existing classes or add other classes besides the system class.
4. In most of the cases, there is only 1 relationship within the same two classes.

Do not include any other text.

##############

The class list is: 

{classes}

#############

The specification text is:

{text}

##############

The association list is:
    
"""

    

prompt_template_plant = """
Step 4
:

You will be asked by the user to create a plant UMl model from a class list and an association list.
Do so in the most clear way possible, avoid class properties and assign molteplicity. 

The format of class list is:

ClassName1, ClassName2, ...

The format of association list is:

example format: mul1 ClassName1 -- mul2 ClassName2

for every class in class list write:

class ClassName1{{}}
class ClassName2{{}}

for every tuple in association list write:

ClassName1 "mul1" -- "mul2" Classname2.

Output plantuml without futher text or explaination. Use tags 
@startuml
at the beginning and 
@enduml 
at the end of the output.

##############

The class list is: 
{classes}

#############

##############

The association list is: 

{association}

#############

The plant uml is:

"""

In [3]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

In [4]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from langchain_ollama import ChatOllama

In [5]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

In [6]:
def build_chian_from_llms(llm):
    
    prompt_noun = PromptTemplate(
         input_variables=["text"],   
         template = prompt_template_noun         
    )
    prompt_class = PromptTemplate(
        input_variables=["text", "noun"],
        template = prompt_template_class                     
    )
    
    prompt_assoc = PromptTemplate(
        input_variables=["text","classes"],
        template = prompt_template_assoc                    
    )
    
    
    prompt_uml = PromptTemplate(
        input_variables=["classes","association"],
        template = prompt_template_plant                
    )

    name_chain  =  prompt_noun | llm | StrOutputParser()
    class_chain = prompt_class | llm | StrOutputParser()
    assoc_chain = prompt_assoc | llm | StrOutputParser()
    uml_chain   =   prompt_uml | llm | StrOutputParser()
    
    full_chain = (
         RunnableLambda(lambda x: {"text": x["text"]})
        |{
            "text": lambda x: x["text"],
            "noun" : name_chain
        }
        | {
            "text": lambda x: x["text"],
            "noun": lambda x: x["noun"],
            "classes": class_chain
        }
        | {
            "text": lambda x: x["text"],
            "classes": lambda x: x["classes"],
            "association": assoc_chain
        }
        | {
            "classes": lambda x: x["classes"],
            "association": lambda x: x["association"],
            "uml": uml_chain
        } 
        | RunnableLambda(lambda x: x["uml"])
        | StrOutputParser()
    )
    return full_chain


In [7]:
import os
from tqdm import tqdm
from docx import Document
# LangChain Python also supports a context manager for tracing a specific block of code.
from langchain_core.tracers.context import tracing_v2_enabled

def process_subfolders_with_chain(root_folder_path, chain, type=''):
    """
    Explores subfolders of the root folder (depth 1), processes each subfolder's `text.txt`
    with the provided LangChain chain, and saves the result in a new file in the same folder.

    Args:
        root_folder_path (str): Path to the root folder.
        chain: A LangChain chain instance to process text inputs.
    """
    for subfolder_name in tqdm(os.listdir(root_folder_path)):
        subfolder_path = os.path.join(root_folder_path, subfolder_name)
        
        # Ensure the current item is a subfolder
        if os.path.isdir(subfolder_path):
            text_file_path = os.path.join(subfolder_path, "text.txt")
            
            # Check if `text.txt` exists in the subfolder
            if not os.path.isfile(text_file_path):
                # If `text.txt` is missing, check for any .docx file in the subfolder
                docx_files = [f for f in os.listdir(subfolder_path) if f.endswith(".docx")]
                if docx_files:
                    docx_file_path = os.path.join(subfolder_path, docx_files[0])
                    # Extract content from the .docx file
                    doc = Document(docx_file_path)
                    text_content = "\n".join([paragraph.text for paragraph in doc.paragraphs])

                    # Save the extracted content to `text.txt`
                    with open(text_file_path, "w", encoding="utf-8") as text_file:
                        text_file.write(text_content)

            # Recheck if `text.txt` now exists
            if os.path.isfile(text_file_path):
                with open(text_file_path, "r", encoding="utf-8") as file:
                    text = file.read()

                # Call the LangChain chain with the input dictionary
                with tracing_v2_enabled():
                    result = chain.invoke({"text": text})

                # Save the result to a new file in the same subfolder
                result_file_path = os.path.join(subfolder_path, f"result_cot2_{type}.txt")
                with open(result_file_path, "w", encoding="utf-8") as result_file:
                    result_file.write(result)

In [8]:
def delete_result_txt_files(root_folder_path):
    """
    Deletes every .txt file that starts with 'result_' in the subfolders of the root folder (depth 1).

    Args:
        root_folder_path (str): Path to the root folder.
    """
    for subfolder_name in os.listdir(root_folder_path):
        subfolder_path = os.path.join(root_folder_path, subfolder_name)
        
        # Ensure the current item is a subfolder
        if os.path.isdir(subfolder_path):
            for file_name in os.listdir(subfolder_path):
                if file_name.startswith("result_cot2_google") and file_name.endswith(".txt"):
                    file_path = os.path.join(subfolder_path, file_name)
                    os.remove(file_path)

In [9]:
#delete_result_txt_files(ROOT_FOLDER)

# COT Open-AI

In [9]:
import os

In [10]:
from dotenv import load_dotenv
assert load_dotenv()

In [11]:
MODEL_OPEN_AI = ["o3-mini", "gpt-4o-mini", "gpt-4o", "gpt-4-turbo", "gpt-3.5-turbo"]

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.agents.chat.output_parser import ChatOutputParser
from IPython.display import display_markdown

In [13]:
def open_ai_cot2(model):
    model_ = ChatOpenAI(model=model)
    chain = build_chian_from_llms(model_)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [14]:
def open_ai_make_example(model):
    model = ChatOpenAI(model=model)
    chain = build_chian_from_llms(model)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res

In [15]:
display_markdown(open_ai_make_example(MODEL_OPEN_AI[0]), raw=True)

@startuml
class InsuranceCompany{}
class Customer{}
class Broker{}
class AccountManager{}
class InsurancePolicy{}
class Contract{}
class Offer{}
class CustomerFile{}
class Invoice{}
class Claim{}
class ClaimCase{}
class Estimator{}
class Report{}
class CompensationDecision{}
class Payment{}
class CustomerAccount{}

Customer "1..1" -- "0..*" Broker.
Customer "1..1" -- "1..1" CustomerFile.
Broker "1..1" -- "0..*" CustomerFile.
Contract "1..1" -- "0..*" AccountManager.
Customer "1..1" -- "0..*" Offer.
Offer "0..1" -- "1..1" Contract.
Contract "1..1" -- "0..*" Invoice.
InsuranceCompany "1..1" -- "0..*" InsurancePolicy.
Contract "1..1" -- "0..*" InsurancePolicy.
Customer "1..1" -- "0..*" Contract.
Customer "1..1" -- "0..*" Claim.
Claim "1..1" -- "1..*" ClaimCase.
ClaimCase "1..1" -- "0..*" Report.
Estimator "1..1" -- "0..*" Report.
ClaimCase "1..1" -- "0..1" CompensationDecision.
CompensationDecision "1..1" -- "1..1" Payment.
Payment "1..1" -- "1..1" CustomerAccount.
@enduml

In [16]:
for model in MODEL_OPEN_AI[0:1]:
    print(f"COT2 with {model}")
    open_ai_cot2(model)

COT with o3-mini


100%|███████████████████████████████████████████| 48/48 [35:20<00:00, 44.17s/it]


## Anthropic

In [12]:
import torch, gc

In [13]:
from langchain_anthropic import ChatAnthropic

In [14]:
MODEL_ANTHROPIC = ["claude-3-7-sonnet-20250219"]

In [15]:
def anthropic_make_example(model):
    model = ChatAnthropic(model=model,temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,)
    chain = build_chian_from_llms(model)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res

In [16]:
def anthropic_cot2(model):
    model_ = ChatAnthropic(model=model)
    chain = build_chian_from_llms(model_)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [17]:
display_markdown(anthropic_make_example(MODEL_ANTHROPIC[0]), raw=True)

@startuml
class InsuranceCompany{}
class Client{}
class InsurancePolicy{}
class Broker{}
class Contract{}
class Offer{}
class Product{}
class Invoice{}
class Claim{}
class ClaimCase{}
class Estimator{}
class Report{}
class CompensationDecision{}
class Payment{}
class Document{}

InsuranceCompany "1..1" -- "0..*" Client
InsuranceCompany "1..1" -- "0..*" Broker
InsuranceCompany "1..1" -- "0..*" InsurancePolicy
InsuranceCompany "1..1" -- "0..*" Product
InsuranceCompany "1..1" -- "0..*" ClaimCase
Client "0..*" -- "1..*" Broker
Client "0..*" -- "0..*" InsurancePolicy
Client "0..*" -- "0..*" Claim
Client "0..*" -- "0..*" Payment
Broker "1..1" -- "0..*" Contract
Broker "1..1" -- "0..*" Offer
InsurancePolicy "1..1" -- "1..1" Contract
Contract "1..1" -- "1..1" Offer
Contract "1..1" -- "0..*" Invoice
Offer "1..1" -- "1..1" Product
Claim "0..*" -- "1..*" ClaimCase
ClaimCase "1..1" -- "0..*" Estimator
ClaimCase "1..1" -- "0..*" Report
ClaimCase "1..1" -- "0..1" CompensationDecision
Estimator "0..*" -- "1..*" Report
CompensationDecision "1..1" -- "0..*" Payment
ClaimCase "0..*" -- "0..*" Document
@enduml

In [19]:
for model in MODEL_ANTHROPIC:
    print(f"COT2 with {model}")
    anthropic_cot2(model)

COT2 with claude-3-7-sonnet-20250219


100%|███████████████████████████████████████████| 48/48 [05:47<00:00,  7.23s/it]


# CoT Open LLMs

## Deepseek

In [22]:
from langchain_deepseek import ChatDeepSeek

In [23]:
MODEL_DEEPSEEK = ["deepseek-chat"] #, "deepseek-reasoner"]

In [24]:
def deepseek_make_example(model):
    model = llm = ChatDeepSeek(
            model=model,
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            # api_key="...",
            # other params...
        )
    chain = build_chian_from_llms(model)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res

In [25]:
def deepseek_cot2(model):
    model_ = ChatDeepSeek(model=model)
    chain = build_chian_from_llms(model_)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [26]:
%%time
display_markdown(deepseek_make_example(MODEL_DEEPSEEK[0]), raw=True)

```plantuml
@startuml
class InsuranceCompany{}
class Client{}
class Broker{}
class InsurancePolicy{}
class Contract{}
class Offer{}
class Coverage{}
class Invoice{}
class Claim{}
class ClaimCase{}
class Accident{}
class MaterialDamage{}
class PhysicalDamage{}
class Estimator{}
class Report{}
class Decision{}
class Refund{}
class Document{}
class Payment{}

InsuranceCompany "1..1" -- "0..*" Client
Client "1..1" -- "0..*" InsurancePolicy
Broker "1..1" -- "0..*" Client
Client "1..1" -- "0..1" Broker
InsurancePolicy "1..1" -- "1..1" Contract
Contract "1..1" -- "0..1" Offer
InsurancePolicy "1..1" -- "1..*" Coverage
InsurancePolicy "1..1" -- "0..*" Invoice
Client "1..1" -- "0..*" Claim
Claim "1..1" -- "1..*" ClaimCase
ClaimCase "1..1" -- "0..1" Accident
Accident "1..1" -- "0..*" MaterialDamage
Accident "1..1" -- "0..*" PhysicalDamage
ClaimCase "1..1" -- "0..*" Estimator
Estimator "1..1" -- "0..*" Report
ClaimCase "1..1" -- "0..1" Decision
Decision "1..1" -- "0..1" Refund
ClaimCase "1..1" -- "0..*" Document
Refund "1..1" -- "0..1" Payment
@enduml
```

CPU times: user 174 ms, sys: 22.1 ms, total: 196 ms
Wall time: 1min 2s


In [27]:
for model in MODEL_DEEPSEEK:
    print(f"COT2 with {model}")
    deepseek_cot2(model)

COT2 with deepseek-chat


100%|███████████████████████████████████████████| 48/48 [28:35<00:00, 35.74s/it]


## Ollama

In [21]:
from langchain_ollama import ChatOllama

In [22]:
MODEL_OLLAMA = [] #["llama3.2:3b-text-fp16"]

In [23]:
def ollama_cot2(model):
    model_ = ChatOllama(
        model=model,
        temperature=0,
    )
    chain = build_chian_from_llms(model_)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [24]:
def ollama_make_example(model):
    model = ChatOllama(
        model=model,
        temperature=0,
    )
    chain = build_chian_from_llms(model)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res
    

In [25]:
#display_markdown(ollama_make_example(MODEL_OLLAMA[0]), raw=True)

In [26]:
for model in MODEL_OLLAMA:
    print(f"COT with {model}")
    ollama_cot2(model)
    gc.collect()
    torch.mps.empty_cache()

## Huggingface

In [27]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace, HuggingFaceEndpoint
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

/Users/marcocalamo/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/marcocalamo/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9DBE5D5C-AC87-30CA-96DA-F5BC116EDA2B> /Users/marcocalamo/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <A51C8C05-245A-3989-8D3C-9A6704422CA5> /Users/marcocalamo/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [28]:
MODEL_HUGGINGFACE = [] #["google/gemma-2-2b-it"] #["Qwen/Qwen2.5-3B-Instruct", "microsoft/Phi-3-mini-4k-instruct", "google/gemma-2-27b-it"]

In [29]:
def huggingface_cot2(model):
    model_id = model
    
    llm = HuggingFaceEndpoint(
        repo_id=model_id,
        task="text-generation",
        max_new_tokens=2048,
        do_sample=False,
        repetition_penalty=1.03,
        temperature=0.01,
    )

    chat = ChatHuggingFace(llm=llm, verbose=True)
    
    chain = build_chian_from_llms(chat)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model_id.replace("/","_"))

In [30]:
def huggingface_make_example(model):

    model_id = model
    
    llm = HuggingFacePipeline.from_model_id(
        model_id=model_id,
        task="text-generation",
        pipeline_kwargs={"temperature": 0.1, "max_new_tokens": 1024}
    )

    chat = ChatHuggingFace(llm=llm, verbose=True)
    
    chain = build_chian_from_llms(chat)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
        As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.
        
        In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 
        
        """})
    return res


In [31]:
#display_markdown(huggingface_make_example(MODEL_HUGGINGFACE[0]), raw=True)

In [32]:
for model in MODEL_HUGGINGFACE:
    print(f"COT2 with {model}")
    huggingface_cot2(model)
    gc.collect()
    torch.mps.empty_cache()

## Mlx-LLM

In [13]:
from langchain_community.llms import MLXPipeline
from langchain_community.chat_models import ChatMLX

In [14]:
MODEL_MLX = ["mlx-community/phi-4-8bit",
             "mlx-community/Falcon3-10B-Instruct-8bit", 
             "mlx-community/Qwen2.5-14B-Instruct-4bit",
             "mlx-community/Mistral-7B-Instruct-v0.3-4bit",
             "mlx-community/DeepSeek-R1-Distill-Qwen-7B-8bit",
             "mlx-community/Llama-3.2-3B-Instruct",
             "mlx-community/gemma-2-9b-8bit",
             #"mlx-community/gemma-2-27b-it-4bit",
            # "mlx-community/Mamba-Codestral-7B-v0.1-8bit",
            #"mlx-community/CodeLlama-13b-Instruct-hf-4bit-MLX"
            ]

In [15]:
def mlx_cot2(model):
    llm = MLXPipeline.from_model_id(
        model_id=model,
        pipeline_kwargs={"max_tokens": 30_000, "temp": 0.1},
    )
    chat = ChatMLX(llm=llm)
    chain = build_chian_from_llms(chat)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model.replace("/","_"))

In [16]:
def mlx_make_example(model):
    llm = MLXPipeline.from_model_id(
        model_id=model,
        pipeline_kwargs={"max_tokens": 3000, "temp": 0.7},
    )
    chat = ChatMLX(llm=llm)
    chain = build_chian_from_llms(chat)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res


In [17]:
display_markdown(mlx_make_example(MODEL_MLX[2]), raw=True)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

@startuml
class AlphaInsurance{}
class Client{}
class Broker{}
class Contract{}
class InsurancePolicy{}
class Profile{}
class HeadOffice{}
class Offer{}
class InsuredEvent{}
class Claim{}
class Case{}
class Estimator{}
class Report{}
class Compensation{}
class Database{}

Client "0..*" -- "1..1" Broker
Client "0..*" -- "0..1" Profile
Client "1..1" -- "0..1" Contract
Broker "0..*" -- "0..*" Contract
Contract "0..*" -- "0..1" InsurancePolicy
Contract "0..*" -- "0..*" Offer
Contract "0..*" -- "0..*" Claim
Claim "0..*" -- "1..1" Case
Case "0..*" -- "0..*" Estimator
Case "0..*" -- "0..1" Compensation
Estimator "0..*" -- "0..*" Report
Report "0..*" -- "0..1" Database
@enduml

In [18]:
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [19]:
import gc, torch
gc.collect()
torch.mps.empty_cache()

In [ ]:
for model in MODEL_MLX:
    print(f"COT2 with {model}")
    mlx_cot2(model)
    import gc, torch
    gc.collect()
    torch.mps.empty_cache()

COT with mlx-community/Mistral-7B-Instruct-v0.3-4bit


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

100%|███████████████████████████████████████████| 44/44 [53:28<00:00, 72.93s/it]


COT with mlx-community/DeepSeek-R1-Distill-Qwen-7B-8bit


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

100%|████████████████████████████████████████| 44/44 [2:40:17<00:00, 218.58s/it]


COT with mlx-community/Llama-3.2-3B-Instruct


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

100%|███████████████████████████████████████████| 44/44 [33:09<00:00, 45.22s/it]


### Clean Cache

In [21]:
import gc, torch
gc.collect()
torch.mps.empty_cache()